In [7]:
pos_words = ['happy', 'fun', 'great', 'exciting', 'fantastic']
neg_words = ['boring', 'disappointing', 'bad', 'unpleasant']

def get_sent(sentence):
    words = sentence.lower().replace('.', '').replace(',','').split()
    pos_score = 0
    neg_score = 0
    for word in words:
        if word in pos_words:
            pos_score += 1
        elif word in neg_words:
            neg_score -= 1
    total_score = pos_score + neg_score
    return total_score
    

sent1 = 'The movie was fun, exciting, but somewhat disappointing.'

score = get_sent(sent1)
print(score)

1


Vader lexicon을 이용한 감성 분석

In [9]:
from nltk import sentiment
#from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
import nltk


sid = SentimentIntensityAnalyzer()

def get_senti_score(text):
    text1 = text.replace('??', '\'') 
    text_sents = tokenize.sent_tokenize(text1)
    sum_desc_com = 0
    for sentence in text_sents:
        ss = sid.polarity_scores(sentence)
        compound = ss['compound']
        sum_desc_com += compound
    try:
        avg_text_com = sum_desc_com/len(text_sents)
    except:
        avg_text_com = 'NA'
    return avg_text_com


sent1 = 'The movie was boring.'
sent2 = 'The movie was fun.'
text1 = 'I watched a newly released movie yesterday. It was an action movie. It was great.'
text2 = "A movie beyond woman equality, history, war, ethics and even religion. This movie was behind it's veil about honor, duty, good, evil, doing what is right, and of course love. I have never seen so profound aspects in another fictitious hero movie. A masterpiece on it's kind."
imdb_review ="Warning! Slight spoiler alert if you are going to watch the movie. Personally, i thought this movie was ridiculous. The entire movie was only tons and tons of filler with nothing interesting except for the three fight scenes, which were all ridiculous. Wonder Woman is to me just a ripoff of Captain America. The whole world war setting, the shield which can deflect bullets and mortar rounds, and how she can just dodge and see bullets coming, was pulled straight from Captain America. As a woman, i am an active feminist and i was glad they finally made a movie where women star in acting and directing roles. However, to me this movie falls just ahead of Catwoman in ratings. Don't waste your time and money with this movie."
amazon_TV_review = "I love OLED. I like this tv so much we ended up getting two of them. I like the smart tv features, and the tv looks great when watching high quality sources (broadcast tv is not high quality). The darks are dark and once you get it calibrated it's a gorgeous picture. Best I've seen."
imdb1 = "Marvel without a doubt, knows the formula to make an visually appealing, fun and well paced superhero movie. There's a lot to love about Spiderman Homecoming, from Tom Holland's take on a teenage Peter Parker (although the tone of his voice grated from time to time) to a solid villain in Vulture, played to perfection by Michael Keaton. There were also some awesome action sequences, my favs being the ferry and the final battle against The Vulture. Sadly, the emotional connection I needed, to make this a truly awesome version of Spidey never came (for me) Raimi's Spiderman 2 hit every note perfectly, and is still in my humble opinion the best Spidey movie out there. While this one looks good, the depth is sorely lacking and with the way Wonder Woman left me feeling so fresh in my head, it was just more obvious to me, that they played it safe and pandered to the easiest base to appease, the Spidey fans, with this one. "
imdb2 = "Now we have Spider-Man Homecoming (2017)!!! OK, true believers; let's begin with, NO ORIGIN STORY! Yes, finally, no more freaking origin story! We ALL know Peter gets bit by a radioactive spider which gives him his powers. DONE! Point scored to fans! Let's move on. Second, Spider-Man Homecoming is still a Sony film but Marvel Studios has MUCH of the creative control when they struck a deal. What this means for fans is, we finally get to see a Spider-Man that we want! ... All in All, this film hits a freaking home run! It's funny, action packed, great writing, true to the character/source material, and leads into some serious Avenger easter eggs! I LOVED THIS version of Spider- Man!! I still love 2002's Spider-Man, but this is leading right into the future of the MCU. EXCELSIOR !!!!! "
ss = sid.polarity_scores(sent2)
compound = ss['compound']
print('sent2: ', compound)

avg_sent_score = get_senti_score(imdb2)
print('avg score: ', avg_sent_score)

sent2:  0.5106
avg score:  0.19871333333333335


SentiWordNet을 이용한 감성 분석

In [6]:
import time
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.corpus import sentiwordnet as swn


def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'): # 형용사
        return 'a'
    elif treebank_tag.startswith('V'): # 동사
        return 'v'
    elif treebank_tag.startswith('N'): # 명사
        return 'n'
    elif treebank_tag.startswith('RB'): # 부사
        return 'r'
    else:
        return 'n'

review = "Personally, i thought this movie was ridiculous. The entire movie was only tons and tons of filler with nothing interesting except for the three fight scenes, which were all ridiculous. Wonder Woman is to me just a ripoff of Captain America. The whole world war setting, the shield which can deflect bullets and mortar rounds, and how she can just dodge and see bullets coming, was pulled straight from Captain America. As a woman, i am an active feminist and i was glad they finally made a movie where women star in acting and directing roles. However, to me this movie falls just ahead of Catwoman in ratings. Don't waste your time and money with this movie."
#-------------------------------------
# Text preprocessing
#-------------------------------------

# 1. 불필요한 symbols과 marks 제거하기
filtered_content = review.replace('!', '').replace(',','').replace('.','').replace('“','').replace('”','').replace('\n','')

# 2. Case conversion; 대문자를 소문자로 바꾸기
filtered_content = filtered_content.lower()

#------------------------------------
# Word tokenization

word_tokens = nltk.word_tokenize(filtered_content)
#-------------------------------------

#------------------------------------------------
# POS tagging
tokens_pos = nltk.pos_tag(word_tokens)

#-----------------------------------------
# Lemmatization

wlem = nltk.WordNetLemmatizer()
lemmatized_tokens = []
for word, pos in tokens_pos:
    #print(word, pos)
    WN_pos = get_wordnet_pos(pos)  #wordnet pos tag로 변환을 해줘야 합니다.
    new_word = wlem.lemmatize(word, WN_pos)
    #print('lemma: ', new_word)
    lemmatized_tokens.append((new_word,WN_pos))
    
pos_score = neg_score = obj_score = token_count = 0

for word, pos in lemmatized_tokens:
    ss_set = None
    ss_set = list(swn.senti_synsets(word, pos))
    if len(ss_set) == 0:
        continue
        
    ss_set = ss_set[0]
    print(word, pos, ss_set.pos_score(), ss_set.neg_score())
    pos_score += ss_set.pos_score()
    neg_score += ss_set.neg_score()
    #obj_score += ss_set.obj_score()
    token_count += 1
    
print('pos_score: {0}, neg_score: {1}'.format(pos_score, neg_score))        
final_score = pos_score - neg_score
print(final_score)
norm_final_score = round(final_score / token_count, 3)
print(norm_final_score) # if norm_final_score >= 0, then it is positive

personally r 0.0 0.0
i a 0.0 0.25
think v 0.0 0.0
movie n 0.0 0.0
be v 0.25 0.125
ridiculous a 0.0 0.625
entire a 0.0 0.0
movie n 0.0 0.0
be v 0.25 0.125
only a 0.0 0.0
ton n 0.0 0.0
ton n 0.0 0.0
filler n 0.0 0.0
nothing n 0.25 0.25
interesting a 0.375 0.0
three n 0.0 0.0
fight n 0.0 0.0
scene n 0.0 0.0
be v 0.25 0.125
ridiculous a 0.0 0.625
woman n 0.0 0.0
be v 0.25 0.125
me n 0.0 0.0
just r 0.0 0.0
a n 0.0 0.0
captain n 0.375 0.0
america n 0.0 0.0
whole a 0.125 0.0
world n 0.0 0.0
war n 0.0 0.0
set v 0.0 0.0
shield n 0.0 0.125
can n 0.0 0.0
deflect v 0.0 0.0
bullet n 0.0 0.0
mortar v 0.0 0.0
round n 0.0 0.0
can n 0.0 0.0
just r 0.0 0.0
dodge v 0.0 0.0
see v 0.0 0.0
bullet n 0.0 0.0
come v 0.0 0.0
be v 0.25 0.125
pull v 0.0 0.0
straight r 0.0 0.0
captain n 0.375 0.0
america n 0.0 0.0
a n 0.0 0.0
a n 0.0 0.0
woman n 0.0 0.0
i n 0.0 0.0
be v 0.25 0.125
an n 0.0 0.125
active a 0.375 0.5
feminist n 0.0 0.0
i n 0.0 0.0
be v 0.25 0.125
glad a 0.125 0.625
finally r 0.0 0.0
make v 0.0 0.0
a 